Oceanography
One of my first forays into academic/ scientific machine learning. It also marks my return to machine learning in my spare time after writing a basic linear algebra library in C++ (to fill the linear algebra gap in my degree) and a taste of computer vision.
I know little of oceanography but it seemed like an interesting first step into helping the scientific/ academic field with machine learning while padding my resume with more real world data.
I always get antsy towards seeing results in data manipulation that I pass over, perhaps, one of the most important parts of the process; data visualization. So, with a break between old habits and new I'm making an effort to explore every part of the data I can before flinging myself into the fun of predictions.

https://www.kaggle.com/competitions/ml-for-oceanography/data

In [1]:
#Imports
import catboost
import math
import numpy as np
import pandas as pd
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GroupShuffleSplit
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import PoissonRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost

Preliminary Data Adjustments
Those data point(s) that need to be changed immediately are noticable based on their inclusion of degrees. Models don't recognize 1-360 as circular unless first displayed as 2D d/180 * pi. 
In a similar way, time parameters must be changed so hh: 2 and mm: 59 is seen as closer to hh: 3 and mm: 0 than hh:2 and mm: 56.

In [2]:
#Preliminary Data Adjustments
def preliminaryFunctions(x):
    #x = x.sort_values(by= ["#YY", "MM", "DD", "hh", "mm"]).reset_index(drop= True)
    x = x.sort_values(by= ["ID"]).reset_index(drop= True)
    x = adjustDegrees(x)
    x = average_windDIR_change(x)  #Worsens by about 0.0002
    x = Wind_Gust_Speed_Consistency(x)
    x = wave_math(x)
    x = DEWP_Diff(x)
    x = Atmo_Pres_Diff(x)
    x = DPD_hour_before(x)  #Boosts score by about 0.0003
    x = wave_and_wind(x)
    x = tons_of_math(x)
    #x = meteorological(x)
    #Below here is just random guessing (0.08866 before (also included some features Catboost considers 0.0 importance))
    #x["wS*wL"] = x["wave_speed_APD"] * x["wave_length_APD"]
    #x["D_ftheta*0.5"] = x["D_ftheta"]**0.5
    #x["waveNum/angFreq"] = x["wave_number"] / x["angular_frequency"]
    #x["Gust_Change_Shift_0**2"] = x["Gust_Change_Shift_0"]**2

    return x

def adjustDegrees(x):   #Changes the degree measure of wind direction and mean wind direction so it's cyclical
    x["WDIR_sin"] = np.sin(x["WDIR(degT)"])   #Wind direction
    x["WDIR_cos"] = np.cos(x["WDIR(degT)"])
    x["MWD_sin"] = np.sin(x["MWD(degT)"])     #Mean wind direction
    x["MWD_cos"] = np.cos(x["MWD(degT)"])
    x["WDIR_Radians"] = (x["WDIR(degT)"] * np.pi) / 180
    x["MWD_Radians"] = (x["MWD(degT)"] * np.pi) / 180
    x.drop(columns=["WDIR(degT)", "MWD(degT)"], inplace= True)
    return x

def DPD_hour_before(x): #Simply the absolute value of the DPD-DPD(hour before)
    #If the DPD from the hour before changes by >=0.5 set to 1
    x["DPD_Diff"] = 0
    x["DPD_Diff"] = (x["DPD(sec)"] - x["DPD(sec)"].shift()) #Changed/ added, removed .abs()
    return x

#If wind direction stays consist (Anywhere around +/-40) hour after hour mark with a 1 (likely changes wave height by a greater amount)
def Wind_Gust_Speed_Consistency(x):
    x["WSPD_Change"] = 0
    x["WSPD_Change"] = (x["WSPD(m/s)"] - x["WSPD(m/s)"].shift())
    x["Gust_Change_Shift_0"] = 0
    x["Gust_Change_Shift_0"] = (x["GST(m/s)"] - x["GST(m/s)"].shift())
    x["Gust_Change_Shift_1"] = 0
    x["Gust_Change_Shift_1"] = (x["GST(m/s)"] - x["GST(m/s)"].shift(1))

    #x["GST/DegreeChange_sin"] = x["GST(m/s)"] / x["drastic_wind_change_sin"]   #About 0.0001 worse
    #x["GST/DegreeChange_cos"] = x["GST(m/s)"] / x["drastic_wind_change_cos"]   #^ same
    return x

#If 0-12 has a lower DEWP by >=0.6 than 13-23 then mark it with a 1
#Maybe try averaging first? Average of 0-12 of same MM,DD,hh is
def DEWP_Diff(x):
    x.set_index("ID", inplace= True)
    x["DEWP_DIFF"] = x["DEWP(degC)"].diff()
    x.reset_index(inplace= True)
    x = x.sort_values(by= "ID").reset_index(drop= True)
    return x

#Difference of the average atmospheric pressure from the day before
#Maybe create a boolean marking 0-12 and 13-23, then average on that and compare to morning/ evening
#Awful, quadruples RMSE
def Atmo_Pres_Diff(x):
    x = x.sort_values(by= "ID").reset_index(drop= True)
    x.set_index("ID", inplace= True)
    x["PRES_DIFF"] = x["PRES(hPa)"].diff()
    x.reset_index(inplace= True)
    x = x.sort_values(by= "ID").reset_index(drop= True)
    return x

def average_windDIR_change(x):
    x["drastic_wind_change_sin"] = (x["MWD_sin"] - x["MWD_sin"].shift()).abs()
    x["drastic_wind_change_cos"] = (x["MWD_cos"] - x["MWD_cos"].shift()).abs()
    return x

def wave_math(x):
    g = 9.8 #Acceleration from gravity 
    #Wave length is calculated with Length = gT^2/2pi where g= 9.8 m/s^2
    #https://datalab.marine.rutgers.edu/ooi-lab-exercises/lab-6-ocean-waves-linking-the-marine-atmosphere-and-the-ocean-surface/lab-6-3/
    #https://rwu.pressbooks.pub/webboceanography/chapter/10-1-wave-basics/
    x["wave_length_APD"] = (g * (x["APD(sec)"]**2)) / (2 * np.pi)
    x["wave_length_DPD"] = (g * (x["DPD(sec)"]**2)) / (2 * np.pi)
    #Wave speed is calculated with wave length / period of the wave
    #Changes depending on the depth, when depth is less than 1/20 wave length the waves "scrape" the floor meaning speed is calculated as sqrt(gd) where d is depth
    #Intermediate (transitional) waves are found at depths between 1/2 and 1/20 that of the wavelength and are calculated as sqrt((gL/2pi)tanh(2pi(d/L))) where L is the wave length
    #https://www.omnicalculator.com/physics/wave-speed
    #x["wave_speed"] = np.sqrt((9.8 * x["wave_length"]) / (2 * np.pi))
    x["wave_speed_APD"] = x["wave_length_APD"] / x["APD(sec)"]
    x["wave_speed_DPD"] = x["wave_length_DPD"] / x["DPD(sec)"]

    #Wave frequency is calculated with wave velocity(speed) / wave length
    #https://www.omnicalculator.com/physics/frequency
    x["wave_frequency_APD"] = x["wave_speed_APD"] / x["wave_length_APD"]
    x["wave_frequency_DPD"] = x["wave_speed_DPD"] / x["wave_length_DPD"]

    #Wave celerity (general speed at which a wave advances/ propagates)
    x["celerity"] = x["wave_length_APD"] * x["wave_frequency_APD"]

    #If a wave height is >=1/7 
    #https://rwu.pressbooks.pub/webboceanography/chapter/10-1-wave-basics/
    #Could surpass the imposed limit given this is an average of the periods and (I assume) the largest wave in the hour
    x["wave_height_limit"] = (x["wave_length_APD"] / 7)

    #Wave Number (units of 1/length)
    #http://www.geo.cornell.edu/ocean/p_ocean/ppt_notes/19_Waves.pdf
    x["wave_number"] = (2 * np.pi) / x["wave_length_APD"]

    #Angular Frequency
    #http://www.geo.cornell.edu/ocean/p_ocean/ppt_notes/19_Waves.pdf
    x["angular_frequency"] = (2 * np.pi) / x["APD(sec)"]    #Adds basically nothing, so I won't include it

    #Dimensionless paramters
    #https://en.m.wikipedia.org/wiki/Wind_wave_model

    return x

def wind_gust_pres_temp_math(x):
    #GST has the highest feature importance (according to Catboost) on it's own (unchanged)
    x["GST_to_Press"] = x["GST(m/s)"] / ["PRES(hPa)"]

    #Doesn't help as much as average_gust_speed but it definitley helps.
    #Considered due to the cause of windspeeds by changes in pressures: https://www.quora.com/How-do-changes-in-atmospheric-pressure-influence-wave-formation-in-the-oceans-and-how-does-this-affect-coastal-erosion
    #CHANGES in pressure, huh? Might look into that more. Change in pressure and the change in windspeed over some time
    x["hPa/WSPD"] = x["WSPD(m/s)"] / x["PRES(hPa)"]

    #From the following two links I'm working under the assumption GST was the max speed the wind blew in the hour and WSPD is the average
    #https://oceanservice.noaa.gov/education/tutorial_currents/media/supp_cur03a.html
    #https://www.reddit.com/r/meteorology/comments/17mn5ke/question_about_sustained_wind_and_wind_gusts/    
    x["WindSPD_div_GustSPD"] = x["WSPD(m/s)"] / x["GST(m/s)"]

def wave_and_wind(x):
    #Just kind of testing stuff now given how much of an influence wind and gust speed play into feature importance
    x["waveSPD/windSPD"] = x["wave_speed_APD"] / x["WSPD(m/s)"]
    x["waveL/windSPD"] = x["wave_length_APD"] / x["WSPD(m/s)"]

    #x["waveLDPD/GST"] = x["wave_length_DPD"] / x["GST(m/s)"]   #Starts overfitting
    #x["waveSpeedDPD/GST"] = x["wave_speed_DPD"] / x["GST(m/s)"] #^

    #x["windSPD/celerity"] = x["WSPD(m/s)"] / x["celerity"]
    #x["GST/celerity"] = x["GST(m/s)"] / x["celerity"]
    x["waveFreq/GST"] = x["wave_frequency_APD"] / x["GST(m/s)"] #Highly favored by feature importance and made for a 0.002 better MSE. Gust has it all
    x["waveSPD_change"] = x["wave_speed_APD"] - x["wave_speed_APD"].shift()
    x["waveLength_change"] = x["wave_length_APD"] - x["wave_length_APD"].shift()

    x["wave_height_limit/(wF/GST)"] = x["wave_height_limit"] / x["waveFreq/GST"]    #Added sub 9
    #x["waveFreq/GST**2"] = x["waveFreq/GST"]**2 #Causes overfitting
    #x["waveFreq/GST_Shift"] = x["waveFreq/GST"] - x["waveFreq/GST"].shift() #Causes overfitting
    
    #https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2023GL104085
    #https://www.sciencedirect.com/topics/engineering/directional-spreading
    #x["omnidirectional_wave_spectrum"] = #S(f)
    #x["estimated_wave_height"] = #Hs
    return x

def meteorological(x):
    #https://earthscience.stackexchange.com/questions/16570/how-to-calculate-relative-humidity-from-temperature-dew-point-and-pressure
    #https://earthscience.stackexchange.com/questions/14899/how-can-temperature-be-calculated-given-relative-humidity-and-dew-point
    x["relative_humidity"] = 100 * np.e * ((243.04 * 17.625 * (x["DEWP(degC)"] - x["ATMP(degC)"])) / ((243.04 + x["ATMP(degC)"]) * (243.04 * x["DEWP(degC)"])))
    return x

#Math, math, and more math
def tons_of_math(x):
    x["APDFreq/DPDFreq"] = x["wave_frequency_APD"] / x["wave_frequency_DPD"]
    #https://agupubs.onlinelibrary.wiley.com/doi/10.1002/2017JC012943
    #Using the Banner (and Donelan et al) method
    x["Beta_s"] = 0
    x["Beta_s"] = np.where(x["APDFreq/DPDFreq"] >= 1.6, 10**(-0.4+0.8393**(-0.567 * np.log((x["APDFreq/DPDFreq"])**2))), x["Beta_s"])
    #x.loc[x["APDFreq/DPDFreq"] >= 1.6, "Beta_s"] = 10**(-0.4+0.8393**(-0.567 * np.log((x["APDFreq/DPDFreq"])**2)))
    x["Beta_s"] = np.where((x["APDFreq/DPDFreq"] >= 0.95) & (x["APDFreq/DPDFreq"] < 1.6), 2.28 * ((x["APDFreq/DPDFreq"])**(-1.3)),x["Beta_s"])
    #x.loc[0.95 <= x["APDFreq/DPDFreq"] <= 1.6, "Beta_s"] = 2.28 * ((x["APDFreq/DPDFreq"])**(-1.3))
    x["Beta_s"] = np.where((x["APDFreq/DPDFreq"] >= 0.56) & (x["APDFreq/DPDFreq"] <= 0.95), 2.61 * ((x["APDFreq/DPDFreq"])**(1.3)),x["Beta_s"])
    #x.loc[0.56 <= x["APDFreq/DPDFreq"] <= 0.95, "Beta_s"] = 2.61 * ((x["APDFreq/DPDFreq"])**(1.3))
    x["x"] = (3 * (x["WDIR_Radians"] - (x["MWD_Radians"] * x["wave_frequency_DPD"])))   #Beta of 3 is used as seen here... https://www.sciencedirect.com/topics/engineering/directional-spreading
    x["cosh"] = ((np.exp(x["x"]) + np.exp(-x["x"])) / 2)
    x["sech"] = ((1 / x["cosh"])**2)    #Forgot to **2 it, if it's worse I might remove it
    x["D_ftheta"] = (0.5 * x["Beta_s"] * x["sech"])   
    #sech = 1 / cosh AND cosh = ((e**x) + (e**(-x))) / 2
    toDrop = ["Beta_s", "wave_length_DPD", "wave_speed_DPD", "wave_frequency_DPD", "WDIR_Radians","MWD_Radians", "x", "cosh", "sech"]    #, "APDFreq/DPDFreq"
    x.drop(columns= toDrop, inplace= True)

    return x


Data Visualization
https://seaborn.pydata.org/
- Number of waves in each direction (wave count paired with degree) (bar chart in polar chart)
- Average height of degree bins (in 45 angles) (bar chart)
- Correlation between wind speed and wave height (scatterplot)
- 

In [3]:
#Visualizations
def total_prepredict_visualizations(x):
    count_of_degrees(x)
    average_height_of_degrees(x)
    wind_speed_wave_height_correlation(x)
    boxplots(x)

def count_of_degrees(x):
    """
        A vast majority of waves are directed between 0-60 degrees.
    """
    #https://python-graph-gallery.com/circular-barplot-basic/
    hist = sns.histplot(x["WDIR(degT)"], bins=np.arange(1,365), kde= False) #Builds histogram of bins (1-365) based on the degrees
    counts = hist.patches   #Counts based on the number of rectangles in the histogram
    degree_counts = np.array([patch.get_height() for patch in counts])  #Builds an array counting the number of degrees in counts
    bins = np.arange(1, 365)    
    plt.close()
    angles = np.deg2rad(bins[:-1])
    fig = plt.figure(figsize= (5,5))
    ax = fig.add_subplot(111, polar= True)
    bars = ax.bar(angles, degree_counts, width= np.deg2rad(1), bottom= 0.0, align= "edge")
    ax.set_xticks(np.deg2rad(np.arange(0,360,30)))
    ax.set_xticklabels([f"{i}" for i in range(0,360,30)])
    plt.title("Degree Counts on Polar Plot")
    plt.show()

def average_height_of_degrees(x):
    #Shows some degree bins experience higher waves than others
    bins = [0, 45, 90, 135, 180, 225, 270, 315, 360]
    labels = ["0-45", "46-90", "91-135", "136-180", "181-225", "226-270", "271-315", "316-360"]
    x["degree_bin"] = pd.cut(x["WDIR(degT)"], bins= bins, labels= labels, right= False, include_lowest= True)
    mean_heights = x.groupby("degree_bin")["WVHT(m)"].mean().reset_index()
    plt.figure(figsize=(5,5))
    ax = sns.barplot(x="degree_bin", y=x["WVHT(m)"], data= mean_heights, palette="viridis")
    plt.xlabel("Direction_Degree_Bins")
    plt.ylabel("Mean_Height")
    plt.title("Average Height of Binned Degrees")
    ax.set_xticklabels(ax.get_xticklabels(), rotation= 40, ha= "right")
    plt.show()
    x.drop(["degree_bin"], axis= 1, inplace= True)

def wind_speed_wave_height_correlation(x):
    """
        Wind speed and gust speed show a similar trend, but wind speed is probably a better measure.
        It's possible having both wind and gust speed in the model could lead it to favor those two as determinants.
        Might create a correlation matrix to see just how similar they are.
    """
    x_axis = x["WVHT(m)"]   #Setting x axis value
    y_axis = x["WSPD(m/s)"] #Setting y axis value
    plt.scatter(x_axis, y_axis) #Generating the scatter plot with the specified values
    b, m = np.polyfit(x_axis, y_axis, 1)    #Fitting a polynomial to the points in x_axis and y_axis (which creates b and m (or a line))
    plt.plot(x_axis, b + m * x_axis, "r-") #Red regression line showing a trend (higher wind speeds tend towards larger waves)
    #Labels, title, and printing
    plt.xlabel("Wave height")
    plt.ylabel("Wind Speed")
    plt.title("Wind Speed and Wave Height Correlation")
    plt.show()

def boxplots(x):
    toShow = ["WSPD(m/s)", "DPD(sec)", "APD(sec)", "PRES(hPa)", "WTMP(degC)", "DEWP(degC)"]
    for i in toShow:
        plt.boxplot(x[i])
        plt.title(f"{i}")
        plt.show()

In [4]:
#Post Data transformation
def postFunctions(x):
    toDrop = ["ID", "WDIR_sin", "MWD_sin", "drastic_wind_change_sin", "Gust_Change_Shift_1", "angular_frequency", "wave_height_limit"] #Usually just ID #Added all but ID due to feature importance of 0 by catboost, will test and assess if score changes much
    x.drop(columns= toDrop, axis= 0, inplace= True)
    x = adjustTime(x)
    x = clean_up_NaNs(x)
    #x = normalization(x)
    return x

def adjustTime(x):  #Changes the year, month, day, hour, and minute parameters so they're cyclical (although, as of now, whether the data is included or not has no affect)
    date_and_time_params = ["#YY", "MM", "DD", "hh"]
    #for i in date_and_time_params:
    #    x[i+"_x"] = (np.sin(2 * np.pi * x[i]) + 1) / 2 #sin_x
    #    x[i+"_y"] = (np.cos(2 * np.pi * x[i]) + 1) / 2 #cos_y
    date_and_time_params.append("mm")     #and minutes is always 50, so both can be dropped. It appears dropping them doesn't change the score much, if at all, but less needless clutter is probably better
    x.drop(columns= date_and_time_params, axis= 0, inplace= True)   #Dropping the non-cyclical versions of these features
    return x

def clean_up_NaNs(x):
    x.replace([np.inf, -np.inf], np.nan, inplace= True)
    x.fillna(value= 0, inplace= True)   #-1 one might be better, right now TST_bestSF has -1 but testing showed 0 was better
    return x

def normalization(x):
    #Is this even necessary?
    x = (x - x.mean()) / x.std()
    return x

In [5]:
#Modeling
def modeling(x_tr, x_te, y_tr):
    model_dictionary = {}
    #model_dictionary["cat"] = catboost.CatBoostRegressor(learning_rate= 0.0001, iterations= 350000, silent= True, loss_function= "RMSE", max_depth= 8, reg_lambda= 15, early_stopping_rounds= 30, bootstrap_type= "Bernoulli")   #0.08627
    model_dictionary["cat"] = catboost.CatBoostRegressor(learning_rate= 0.0002, iterations= 15000, silent= True, loss_function= "RMSE", max_depth= 8, reg_lambda= 11, early_stopping_rounds= 25, bootstrap_type= "Bernoulli")   #0.08627
    model_dictionary["cat_two"] = catboost.CatBoostRegressor(learning_rate= 0.0001, iterations= 65000, silent= True, loss_function= "RMSE", max_depth= 9, reg_lambda= 7, early_stopping_rounds= 30)
    model_dictionary["lr"] = LinearRegression()  #0.09487
    model_dictionary["pr"] = PoissonRegressor(max_iter= 10000)    #0.1140
    model_dictionary["dtr"] = DecisionTreeRegressor(max_depth= 8, min_samples_split= 10, max_features= "sqrt") #0.1378
    model_dictionary["svrr"] = SVR(kernel= "linear")  #0.1775
    model_dictionary["r"] = Ridge(alpha= 10, fit_intercept= False, copy_X= True, solver= "auto", tol= 0.0001) #0.09490
    model_dictionary["xgbr"] = xgboost.XGBRegressor(n_estimators= 15000, learning_rate= 0.0002, booster= "gbtree", subsample= 0.8, max_depth= 8, gamma= 0.35)   #0.09044
    #model_dictionary["xgbr"] = xgboost.XGBRegressor(n_estimators= 350000, learning_rate= 0.0001, booster= "gbtree", subsample= 0.8, max_depth= 8, gamma= 0.35)   #0.09044
    model_dictionary["las"] = Lasso(alpha= 0.05)    #0.1078
    model_dictionary["net"] = ElasticNet(alpha= 0.5, max_iter= 15000, tol= 0.000005)    #0.1079
    model_dictionary["knn"] = KNeighborsRegressor(n_neighbors= 11, leaf_size= 200, weights= "uniform", algorithm= "brute")
    model_dictionary["gbr"] = GradientBoostingRegressor(learning_rate= 0.0002, loss= "squared_error", n_estimators= 15000, criterion= "squared_error", max_depth= 8, n_iter_no_change= 30) #0.11
    
    
    stacked = 1
    model_list = ["cat", "xgbr", "knn", "lr", "dtr", "gbr", "net", "r"] #
    #If you want to go with weighted average for score: cat 0.15, xgbr 0.5, knn 0.25, lr 1.15, dtr 0.2, net 0.0
    """
    Un-tuned cat, xgbr, lr, r: TEST-? REAL-0.09447
    cat, xgbr: TEST-0.0877 REAL-0.08859
    cat, xgbr, r(f_i: TRUE): TEST-0.086375 REAL-0.08899
    cat, xgbr: TEST-0.0866 REAL-0.08907
    cat (with DTheta feature): TEST-0.08530 REAL-0.08737
    cat (with DTheta and it's parts): TEST-0.08453 REAL-0.08901 (WAY OVERFITS)
    cat, xgbr (just DTheta): TEST-0.086 REAL- 0.08772   (I'd like to try just Cat with some other formulas in the article DTheta was found)
    """
    if(stacked):
        models_pred_df_tr = pd.DataFrame()
        #https://towardsdatascience.com/simple-model-stacking-explained-and-automated-1b54e4357916
        #Runs each model in the model_list array and adds it's prediction to a new dataframe
        splits = 10
        for model in model_list:
            fold = KFold(n_splits= splits, shuffle= True, random_state= 42)  #KFold with seed
            k_fold_main_series = pd.Series()  #Creating empty series that predictions will fall into
            #Fit and predict based on kfold count
            iteration = 0
            for train_x, test_x in fold.split(x_tr, y_tr):
                new_train_x, new_test_x = x_tr.iloc[train_x], x_tr.iloc[test_x]
                new_train_y, new_test_y = y_tr.iloc[train_x], y_tr.iloc[test_x]
                model_dictionary[model].fit(new_train_x, new_train_y)
                prediction = model_dictionary[model].predict(new_test_x)
                #Combine all predicitions into one series
                k_fold_iter_series = pd.Series(prediction)
                k_fold_iter_series = k_fold_iter_series.set_axis(new_test_x.index)
                k_fold_main_series = pd.concat([k_fold_main_series, k_fold_iter_series])
                iteration += 1
                print(f"Finished train {model} ({iteration}/{splits})")
            models_pred_df_tr[f"{model}_preds"] = k_fold_main_series
        print("\n*** Finished training ***\n")

        #Creating new dataframe with test features
        models_pred_df_te = pd.DataFrame()
        for model in model_list:
            model_dictionary[model].fit(x_tr, y_tr)
            prediction = model_dictionary[model].predict(x_te)
            models_pred_df_te[f"{model}_preds"] = pd.Series(prediction)
            print(f"Finished test {model}")
        print("\n*** Finished test ***\n")
        
        #Merging dataframes with primary features
        #models_pred_df_tr = pd.merge(models_pred_df_tr, x_tr, left_index= True, right_index= True)
        #models_pred_df_te = pd.merge(models_pred_df_te, x_te, left_index= True, right_index= True)

        models_pred_df_tr.sort_index(inplace= True)
        models_pred_df_te.sort_index(inplace= True)
        #Meta-model selection and prediction
        meta_model = "lr"
        print(f"\nPredicting with meta-model {meta_model}\n")
        model_dictionary[meta_model].fit(models_pred_df_tr, y_tr)
        final_prediction = model_dictionary[meta_model].predict(models_pred_df_te)
        #models_pred_df_tr.to_csv("./train_meta_model_features.csv")
        #y_tr.to_frame().to_csv("./train_meta_model_real.csv")
        #models_pred_df_te.to_csv("./test_meta_model_features.csv")

    else:
        predicts_list = []
        catBoostFeatureImportance = 0
        for model in model_list:
            model_dictionary[model].fit(x_tr, y_tr)
            single_prediction = model_dictionary[model].predict(x_te)
            predicts_list.append(single_prediction)
            if(catBoostFeatureImportance):
                featureImp = pd.DataFrame()
                featureImp["features"] = x_tr.columns
                featureImp["importance"] = model_dictionary[model].feature_importances_
                featureImp.to_csv("./CatFeatureImp.csv", index=False)
                print(model_dictionary[model].feature_importances_)
        final_prediction = np.mean(np.asarray(predicts_list), axis= 0)
    
    
    return final_prediction
    #print(model.feature_importances_)   #Higher number means it'll have a bigger impact on the prediction when it moves
    #https://machinelearningmastery.com/stacking-ensemble-machine-learning-with-python/
    

In [6]:
#Train Test Split
def TT_Split(x):
    #https://stackoverflow.com/questions/61337373/split-on-train-and-test-separating-by-group
    #Must split taking time into consideration
    #x = x.sort_values(by= ["#YY", "MM", "DD", "hh", "mm"]).reset_index(drop= True)
    x = x.sort_values(by= ["ID"]).reset_index(drop= True)

    """
    #split = int(len(x)*0.605) #Using this ~0.605 and ~0.305 split because it's reflective of the actual train len(3886) and test (1535) (eq. ~0.3950)
    #x_tr = x.copy().iloc[:split]
    #x_te = x.copy().iloc[split:]
    #y_tr = x_tr["WVHT(m)"]
    #x_tr.drop(columns= ["WVHT(m)"], inplace= True)
    #y_te = x_te["WVHT(m)"]
    #x_te.drop(columns= ["WVHT(m)"], inplace= True)
    """
    x["temp_date"] = x["#YY"].astype(str) + "-" + x["MM"].astype(str) + "-" + x["DD"].astype(str)
    y = x["WVHT(m)"].copy()
    gss = GroupShuffleSplit(n_splits= 1, train_size= 0.6050, random_state= 3)
    train_ix, test_ix = next(gss.split(X= x, y= y, groups= x["temp_date"]))
    x.drop(columns= ["temp_date", "WVHT(m)"], inplace= True)
    x_tr = x.loc[train_ix]  #Training features
    y_tr = y.loc[train_ix]  #Training values
    x_te = x.loc[test_ix]   #Testing data
    y_te = y.loc[test_ix]   #Testing values (what's being predicted)

    print(f"x: {len(x)} \nx_tr: {len(x_tr)} \nx_te: {len(x_te)}")
    return x_tr, x_te, y_tr, y_te

In [7]:
#Using Train
def using_train(train):
    #Train split and test
    x_tr, x_te, y_tr, y_te = TT_Split(train)   #Making a copy of df_tr because dropping the height column in the TT_Split affects the pointed to version of df_tr
    #Preliminary data prep
    x_tr = preliminaryFunctions(x_tr)
    x_te = preliminaryFunctions(x_te)
    #Secondary data prep
    x_tr = postFunctions(x_tr)
    x_te = postFunctions(x_te)
    #print(x_tr.isnull().any())
    #Test
    prediction = modeling(x_tr, x_te, y_tr)
    prediction = np.around(prediction, decimals= 2)

    MSE = mean_squared_error(y_te, prediction, squared= False)

    print(x_tr.columns)
    print(MSE)

    x_tr.to_csv("./train_features.csv")
    y_te = pd.Series(y_te)
    y_te = y_te.reset_index(drop= True)
    prediction = pd.Series(prediction)
    x_te = x_te.merge(y_te.rename("REAL"), how= "outer", left_index= True, right_index= True)
    x_te = x_te.merge(prediction.rename("PREDICTS"), how= "outer", left_index= True, right_index= True)
    x_te["DIFFERENCE"] = np.abs(x_te["REAL"] - x_te["PREDICTS"])
    x_te.to_csv("./train_split_test_primary_features.csv")
    
    return "Test/ Train Prediction Complete"

In [8]:
#Using Test
def using_test(train, test):
    test_IDs = pd.DataFrame(test["ID"].copy())
    train = preliminaryFunctions(train)
    test = preliminaryFunctions(test)
    train_y = train["WVHT(m)"].copy()
    train.drop(columns= ["WVHT(m)"], inplace= True)
    submission = pd.DataFrame(test["ID"].copy())
    train = postFunctions(train)
    test = postFunctions(test)

    prediction = modeling(train, test, train_y)
    prediction = pd.Series(prediction)
    prediction = prediction.round(2)
    submission = submission.merge(prediction.rename("WVHT(m)"), how= "outer", left_index= True, right_index= True)

    print("Train:", len(train))
    print("Test:", len(test))

    submission.to_csv("./test_sub_11.csv", index= False)
    
    return "Test Prediction Complete"

In [9]:
#Main
#Loading in the data
df_tr = pd.read_csv("/Users/nwallace/Desktop/MiscCoding/PythonML/Oceana Oceanography/train.csv")
df_te = pd.read_csv("/Users/nwallace/Desktop/MiscCoding/PythonML/Oceana Oceanography/test.csv")
#Visuals
#total_prepredict_visualizations(df_tr)
#Pulling test from train
#using_train(df_tr.copy())
#Using all of train and the actual test
using_test(df_tr, df_te)

Finished train cat (1/10)
Finished train cat (2/10)
Finished train cat (3/10)
Finished train cat (4/10)
Finished train cat (5/10)
Finished train cat (6/10)
Finished train cat (7/10)
Finished train cat (8/10)
Finished train cat (9/10)
Finished train cat (10/10)
Finished train xgbr (1/10)
Finished train xgbr (2/10)
Finished train xgbr (3/10)
Finished train xgbr (4/10)
Finished train xgbr (5/10)
Finished train xgbr (6/10)
Finished train xgbr (7/10)
Finished train xgbr (8/10)
Finished train xgbr (9/10)
Finished train xgbr (10/10)
Finished train knn (1/10)
Finished train knn (2/10)
Finished train knn (3/10)
Finished train knn (4/10)
Finished train knn (5/10)
Finished train knn (6/10)
Finished train knn (7/10)
Finished train knn (8/10)
Finished train knn (9/10)
Finished train knn (10/10)
Finished train lr (1/10)
Finished train lr (2/10)
Finished train lr (3/10)
Finished train lr (4/10)
Finished train lr (5/10)
Finished train lr (6/10)
Finished train lr (7/10)
Finished train lr (8/10)
Finishe

'Test Prediction Complete'

Notes:
    - Wave breaks occur if the wave height becomes larger than 1/7 the wave length (becomes too steep), so height prediction can't exceed 1/7 length

Sites:
    - https://rwu.pressbooks.pub/webboceanography/chapter/10-1-wave-basics/
    - https://www.ndbc.noaa.gov/faq/wavecalc.shtml
    - https://cdip.ucsd.edu/m/documents/wave_measurement.html
    - https://royalsocietypublishing.org/doi/10.1098/rsta.2017.0087
    - https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5948875/
    - https://journals.ametsoc.org/view/journals/wefo/33/4/waf-d-17-0186_1.xml
    - https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2023GL104085
    - 